In [16]:
import sys
import itertools
import pandas as pd
from dotenv import dotenv_values
from pathlib import Path

config = dotenv_values("./../../config/.env") # take environment variables from .env.
base_path = Path(config["BASE_PATH"])
sys.path.append(str(base_path/"code"))

In [ ]:

import pandas as pd
from dotenv import dotenv_values
from pathlib import Path

config = dotenv_values("./../../config/.env") # take environment variables from .env.
base_path = Path(config["BASE_PATH"])


In [7]:
annotations = pd.read_pickle(base_path/"data/evaluation/Eval-LLMs.pkl")

In [10]:
annotations.groupby(["report_id"]).count()

,paragraph_nr,text,n_words,labeled,cohere_loss,cohere_unexpected,openai_loss,openai_unexpected
report_id,,,,,,,,
3585,39,39,39,39,39,39,0,0
3935,1,1,1,1,1,1,0,0
4123,157,157,157,157,157,157,0,0
5110,382,382,382,382,382,382,0,0
5407,184,184,184,184,184,184,0,0
BaloiseHoldingLtd-AR_2015,934,934,934,934,934,934,0,0
BeazleyPLC-AR_2017,1069,1069,1069,1069,1069,1069,0,0


In [13]:
print("Cohere:")
print("loss:      ", annotations.cohere_loss.apply(lambda x: x == "True").sum())
print("unexpected:", annotations.cohere_unexpected.apply(lambda x: x == "True").sum())

Cohere:
loss:       314
unexpected: 195


Create multiple LaTeX tables, save them and print out the input statement to be used for compiling the LaTeX Document

In [17]:
df = pd.read_pickle(base_path/"data/labeling/active-learning-iteration-2.pkl")
df_l = df[df.labeled]
df_l.strategy.value_counts()
df_l["sampling-strategy"] = df_l.strategy.apply(lambda x: "least confidence" if x.split("_")[0] == "lc" else "$\mathcal{F}_{+}$")
df_l["class"] = df_l.strategy.apply(lambda x: x.split("_")[1] if x.split("_")[0] == "lc" else x.split("_")[2])
df_l["iteration"] = df_l.strategy.apply(lambda x: int(x.split("_")[-1])+1 if len(x.split("_")) > 3 else 1)

table_path = base_path/"writing"/"MSc-Thesis-Emerging-Risks"/"tables"/"active-learning-stats"
groups = ["sampling-strategy", "iteration", "class"]
for i in range(len(groups)):
    for g in list(itertools.permutations(groups, i+1)):
        g = list(g)
        if i == 0:
            capt = "Overview of active-learning samples grouped by "+", ".join(g)+".",
        else:
            capt = "Overview of active-learning samples grouped by "+", ".join(g[:-1])+" and "+ g[-1]+".",
        t = df_l.groupby(g).agg({"labeled": "count", "loss": "sum", "unexpected": "sum"}).rename(columns={"labeled": "count"})
        t.loss = t.loss.astype(str) + t.loss.div(t["count"]).mul(100).apply(lambda x: f""" ({x:.1f} \\%)""")
        t.unexpected = t.unexpected.astype(str) + t.unexpected.div(t["count"]).mul(100).apply(lambda x: f""" ({x:.1f} \\%)""")
        t.rename(columns={"labeled": "count"}).style.format().to_latex(
            table_path/str("_".join(g)+".tex"),
            position="H",
            caption=capt[0],
            label="tab:al_overview_"+"_".join(g),
            environment="longtable",
            sparse_index=True,
            sparse_columns=True,
            hrules=True)
        print("\input{tables/active-learning-stats/"+"_".join(g)+".tex}")


\input{tables/active-learning-stats/sampling-strategy.tex}
\input{tables/active-learning-stats/iteration.tex}
\input{tables/active-learning-stats/class.tex}
\input{tables/active-learning-stats/sampling-strategy_iteration.tex}
\input{tables/active-learning-stats/sampling-strategy_class.tex}
\input{tables/active-learning-stats/iteration_sampling-strategy.tex}
\input{tables/active-learning-stats/iteration_class.tex}
\input{tables/active-learning-stats/class_sampling-strategy.tex}
\input{tables/active-learning-stats/class_iteration.tex}
\input{tables/active-learning-stats/sampling-strategy_iteration_class.tex}
\input{tables/active-learning-stats/sampling-strategy_class_iteration.tex}
\input{tables/active-learning-stats/iteration_sampling-strategy_class.tex}
\input{tables/active-learning-stats/iteration_class_sampling-strategy.tex}
\input{tables/active-learning-stats/class_sampling-strategy_iteration.tex}
\input{tables/active-learning-stats/class_iteration_sampling-strategy.tex}


/tmp/ipykernel_65226/1901378243.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l["sampling-strategy"] = df_l.strategy.apply(lambda x: "least confidence" if x.split("_")[0] == "lc" else "$\mathcal{F}_{+}$")
/tmp/ipykernel_65226/1901378243.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l["class"] = df_l.strategy.apply(lambda x: x.split("_")[1] if x.split("_")[0] == "lc" else x.split("_")[2])
/tmp/ipykernel_65226/1901378243.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [22]:
df = pd.read_pickle(base_path/"data/labeling/GT.pkl")
tf = df[(df.labeled == True) & (df.strategy == "sequential") ]
tf

,report_id,paragraph_nr,text,n_words,filing_type,labeled,strategy,loss,unexpected,company,year
345824,3585,513,The corporate headquarters of the Company is l...,42,10K,True,sequential,0,0,ASSURANCEAMERICA CORP,2007.0
345825,3585,530,The Company reported net income of $.3 million...,108,10K,True,sequential,0,0,ASSURANCEAMERICA CORP,2007.0
345826,3585,531,Contributing factors towards the Company’s 200...,176,10K,True,sequential,1,0,ASSURANCEAMERICA CORP,2007.0
345827,3585,534,Gross premiums written for the year ended Dece...,238,10K,True,sequential,0,0,ASSURANCEAMERICA CORP,2007.0
345828,3585,535,Premiums written refers to the total amount of...,139,10K,True,sequential,0,0,ASSURANCEAMERICA CORP,2007.0
...,...,...,...,...,...,...,...,...,...,...,...
1150793,BeazleyPLC-AR_2017,2301,Sidecar special purpose syndicate Specialty re...,67,annual_report,True,sequential,0,0,BeazleyPLC,2017.0
1150794,BeazleyPLC-AR_2017,2305,Total shareholder return (TSR) The increase in...,25,annual_report,True,sequential,0,0,BeazleyPLC,2017.0
1150795,BeazleyPLC-AR_2017,2307,Unearned premiums reserve The portion of premi...,25,annual_report,True,sequential,0,0,BeazleyPLC,2017.0
1150796,BeazleyPLC-AR_2017,2309,If you have finished reading this report and n...,31,annual_report,True,sequential,0,0,BeazleyPLC,2017.0


In [25]:
tf.to_pickle(base_path/"data/evaluation/GT.pkl")

In [26]:
tf.groupby(["filing_type", "report_id"]).agg({"labeled": "count", "loss": "sum", "unexpected": "sum"}).rename(columns={"labeled": "count"})

count loss unexpected
filing_type   report_id                                       
10K           3585                          39    3          0
              3935                           1    0          0
              4123                         157   29          4
              5110                         382   27         11
              5407                         184   10          0
annual_report BaloiseHoldingLtd-AR_2015    934   19          4
              BeazleyPLC-AR_2017          1069   49          2

Print the text of the reoprt with only one paragraph

In [ ]:
tf[tf.report_id =="3935"].text.values[0]